# Background & Objectives
The goal of this challenge is to query the database from our Python code.

# Data
We will work with the ecommerce.sqlite database available at this URL:
https://wagon-public-datasets.s3.amazonaws.com/sql_databases/ecommerce.sqlite

# Investigating the database

## Creating the connection and cursor

In [1]:
import sqlite3
import pandas as pd

connection = sqlite3.connect('data/ecommerce.sqlite')
connection.row_factory = sqlite3.Row
cursor = connection.cursor()

## Finding the tables in the database

In [2]:
def database_explorer(db_cursor):
    """ Shows the number of tables in the database"""

    query = """
            SELECT name FROM sqlite_master  
            WHERE type='table';
            """
    db_cursor.execute(query)
    rows = db_cursor.fetchall()

    print(f"""
    {type(rows) = }
    {len(rows) = }
    """)

    return [row[row.keys()[0]] for row in rows]

database_explorer(db_cursor=cursor)


    type(rows) = <class 'list'>
    len(rows) = 5
    


['Customers', 'Shippers', 'Employees', 'Orders', 'OrderDetails']

## Displaying the whole orders table

In [3]:
def query_orders(db_cursor):
    """return a list of orders displaying each column"""
    query = '''
            SELECT *
            FROM orders
            '''
    db_cursor.execute(query)
    rows = db_cursor.fetchall()
    print(f"""
{type(rows) = } / {len(rows) = }
{type(rows) = }
{[row.keys()for row in rows][0] = }
""")

    orders_table_keys = [row.keys() for row in rows][0]

    for index, row in enumerate(rows[::]):
        print(f"""
CHECKING ROW CONTENT | row_{index + 1}:""")
        for key in orders_table_keys:
            print(f"""{key}: {row[key]}""")

    return rows

query_orders(db_cursor=cursor)


type(rows) = <class 'list'> / len(rows) = 20
type(rows) = <class 'list'>
[row.keys()for row in rows][0] = ['OrderID', 'CustomerID', 'EmployeeID', 'OrderDate', 'RequiredDate', 'ShippedDate', 'ShipVia', 'FreightCharge']


CHECKING ROW CONTENT | row_1:
OrderID: 1
CustomerID: 1
EmployeeID: 1
OrderDate: 2012-01-04
RequiredDate: 2012-01-09
ShippedDate: 2012-01-05
ShipVia: 1
FreightCharge: 3.75

CHECKING ROW CONTENT | row_2:
OrderID: 2
CustomerID: 2
EmployeeID: 2
OrderDate: 2012-01-27
RequiredDate: 2012-02-01
ShippedDate: 2012-01-28
ShipVia: 1
FreightCharge: 7.25

CHECKING ROW CONTENT | row_3:
OrderID: 3
CustomerID: 4
EmployeeID: 1
OrderDate: 2012-02-19
RequiredDate: 2012-02-24
ShippedDate: 2012-02-23
ShipVia: 2
FreightCharge: 5.5

CHECKING ROW CONTENT | row_4:
OrderID: 4
CustomerID: 2
EmployeeID: 4
OrderDate: 2012-03-13
RequiredDate: 2012-03-18
ShippedDate: 2012-03-14
ShipVia: 2
FreightCharge: 13.5

CHECKING ROW CONTENT | row_5:
OrderID: 5
CustomerID: 4
EmployeeID: 2
OrderDate: 2012-04-05
R

## Displaying order information for orders in a specific time window

In [4]:
def get_orders_range(db_cursor, date_from, date_to):
    """ return a list of orders displaying all columns with OrderDate between
    date_from and date_to (excluding date_from and including date_to)"""

    query = '''
            SELECT *
            FROM orders
            WHERE OrderDate > ?
            AND OrderDate <= ?
            '''
    db_cursor.execute(query, (date_from, date_to))
    rows = db_cursor.fetchall()

    print(f"""
{type(rows) = } / {len(rows) = }
{type(rows) = }
{[row.keys()for row in rows][0] = }
""")

    orders_table_keys = [row.keys() for row in rows][0]

    for index, row in enumerate(rows[::]):
        print(f"""
CHECKING ROW CONTENT | row_{index + 1}:""")
        for key in orders_table_keys:
            print(f"""{key}: {row[key]}""")
    return rows

get_orders_range(db_cursor= cursor, date_from= "2012-04-06", date_to="2012-09-04") 



type(rows) = <class 'list'> / len(rows) = 6
type(rows) = <class 'list'>
[row.keys()for row in rows][0] = ['OrderID', 'CustomerID', 'EmployeeID', 'OrderDate', 'RequiredDate', 'ShippedDate', 'ShipVia', 'FreightCharge']


CHECKING ROW CONTENT | row_1:
OrderID: 6
CustomerID: 3
EmployeeID: 3
OrderDate: 2012-04-28
RequiredDate: 2012-05-03
ShippedDate: 2012-04-29
ShipVia: 2
FreightCharge: 11.0

CHECKING ROW CONTENT | row_2:
OrderID: 7
CustomerID: 4
EmployeeID: 3
OrderDate: 2012-05-21
RequiredDate: 2012-05-26
ShippedDate: 2012-05-22
ShipVia: 1
FreightCharge: 11.25

CHECKING ROW CONTENT | row_3:
OrderID: 8
CustomerID: 1
EmployeeID: 4
OrderDate: 2012-06-13
RequiredDate: 2012-06-18
ShippedDate: 2012-06-14
ShipVia: 4
FreightCharge: 13.5

CHECKING ROW CONTENT | row_4:
OrderID: 9
CustomerID: 2
EmployeeID: 1
OrderDate: 2012-07-06
RequiredDate: 2012-07-11
ShippedDate: 2012-07-07
ShipVia: 3
FreightCharge: 4.75

CHECKING ROW CONTENT | row_5:
OrderID: 10
CustomerID: 3
EmployeeID: 2
OrderDate: 2012-07-29

## displaying the number of days between OrderDate and ShippedDate

In [5]:
def get_waiting_time(db_cursor):
    """get a list with all the orders displaying each column
       and calculate an extra TimeDelta column displaying the number of days
       between OrderDate and ShippedDate, ordered by ascending TimeDelta"""

    query = '''
            SELECT
            *,
            julianday(orders.ShippedDate) - julianday(orders.OrderDate) AS TimeDelta
            FROM orders
            ORDER BY TimeDelta
            '''
    db_cursor.execute(query)
    rows = db_cursor.fetchall()

    print(f"""
{type(rows) = } / {len(rows) = }
{type(rows) = }
{[row.keys()for row in rows][0] = }
""")

    orders_table_keys = [row.keys() for row in rows][0]

    for index, row in enumerate(rows[::]):
        print(f"""
CHECKING ROW CONTENT | row_{index + 1}:""")
        for key in orders_table_keys:
            print(f"""{key}: {row[key]}""")
    return rows

get_waiting_time(db_cursor=cursor)


type(rows) = <class 'list'> / len(rows) = 20
type(rows) = <class 'list'>
[row.keys()for row in rows][0] = ['OrderID', 'CustomerID', 'EmployeeID', 'OrderDate', 'RequiredDate', 'ShippedDate', 'ShipVia', 'FreightCharge', 'TimeDelta']


CHECKING ROW CONTENT | row_1:
OrderID: 1
CustomerID: 1
EmployeeID: 1
OrderDate: 2012-01-04
RequiredDate: 2012-01-09
ShippedDate: 2012-01-05
ShipVia: 1
FreightCharge: 3.75
TimeDelta: 1.0

CHECKING ROW CONTENT | row_2:
OrderID: 2
CustomerID: 2
EmployeeID: 2
OrderDate: 2012-01-27
RequiredDate: 2012-02-01
ShippedDate: 2012-01-28
ShipVia: 1
FreightCharge: 7.25
TimeDelta: 1.0

CHECKING ROW CONTENT | row_3:
OrderID: 4
CustomerID: 2
EmployeeID: 4
OrderDate: 2012-03-13
RequiredDate: 2012-03-18
ShippedDate: 2012-03-14
ShipVia: 2
FreightCharge: 13.5
TimeDelta: 1.0

CHECKING ROW CONTENT | row_4:
OrderID: 5
CustomerID: 4
EmployeeID: 2
OrderDate: 2012-04-05
RequiredDate: 2012-04-10
ShippedDate: 2012-04-06
ShipVia: 3
FreightCharge: 8.75
TimeDelta: 1.0

CHECKING ROW CONTE

## Obtaining  a list of all orders (order_id, customer.contact_name, employee.firstname)

In [6]:
def detailed_orders(db_cursor):
    """return a list of all orders (order_id, customer.contact_name,
    employee.firstname) ordered by order_id"""
    query = '''
            SELECT
            orders.OrderID,
            customers.ContactName,
            employees.FirstName
            FROM orders
            JOIN customers ON orders.CustomerID = customers.CustomerID
            JOIN employees ON orders.EmployeeID = employees.EmployeeID
            ORDER BY orders.OrderID
            '''
    rows = db_cursor.execute(query).fetchall()
    print(f"""
{type(rows) = } / {len(rows) = }
{type(rows) = }
{[row.keys()for row in rows][0] = }
""")

    orders_table_keys = [row.keys() for row in rows][0]

    for index, row in enumerate(rows[::]):
        print(f"""
CHECKING ROW CONTENT | row_{index + 1}:""")
        for key in orders_table_keys:
            print(f"""{key}: {row[key]}""")
    return rows

detailed_orders(db_cursor=cursor)


type(rows) = <class 'list'> / len(rows) = 20
type(rows) = <class 'list'>
[row.keys()for row in rows][0] = ['OrderID', 'ContactName', 'FirstName']


CHECKING ROW CONTENT | row_1:
OrderID: 1
ContactName: Dick Terrcotta
FirstName: James

CHECKING ROW CONTENT | row_2:
OrderID: 2
ContactName: Barbara Berber
FirstName: Patty

CHECKING ROW CONTENT | row_3:
OrderID: 3
ContactName: Toni Faucet
FirstName: James

CHECKING ROW CONTENT | row_4:
OrderID: 4
ContactName: Barbara Berber
FirstName: Lisa

CHECKING ROW CONTENT | row_5:
OrderID: 5
ContactName: Toni Faucet
FirstName: Patty

CHECKING ROW CONTENT | row_6:
OrderID: 6
ContactName: Jim Wood
FirstName: Robert

CHECKING ROW CONTENT | row_7:
OrderID: 7
ContactName: Toni Faucet
FirstName: Robert

CHECKING ROW CONTENT | row_8:
OrderID: 8
ContactName: Dick Terrcotta
FirstName: Lisa

CHECKING ROW CONTENT | row_9:
OrderID: 9
ContactName: Barbara Berber
FirstName: James

CHECKING ROW CONTENT | row_10:
OrderID: 10
ContactName: Jim Wood
FirstName: Patty



## Obtaining  the total amount spent per customer

In [7]:
def spent_per_customer(db_cursor):
    """return the total amount spent per customer ordered by ascending total
       amount (to 2 decimal places)
       Example :
        Jean   |   100
        Marc   |   110
        Simon  |   432
        ..."""
    
    query = '''
            SELECT
            Customers.ContactName,
            ROUND(SUM(details.UnitPrice * details.Quantity)) AS cumulative_amount
            FROM OrderDetails AS details
            JOIN Orders ON details.OrderID = Orders.OrderId
            JOIN Customers ON Orders.CustomerID = Customers.CustomerID
            GROUP BY ContactName
            ORDER BY cumulative_amount
            '''
    rows = db_cursor.execute(query).fetchall()
    print(f"""
{type(rows) = } / {len(rows) = }
{type(rows) = }
{[row.keys()for row in rows][0] = }
""")

    orders_table_keys = [row.keys() for row in rows][0]

    for index, row in enumerate(rows[::]):
        print(f"""
CHECKING ROW CONTENT | row_{index + 1}:""")
        for key in orders_table_keys:
            print(f"""{key}: {row[key]}""")

    return rows

spent_per_customer(db_cursor= cursor)


type(rows) = <class 'list'> / len(rows) = 5
type(rows) = <class 'list'>
[row.keys()for row in rows][0] = ['ContactName', 'cumulative_amount']


CHECKING ROW CONTENT | row_1:
ContactName: Jim Wood
cumulative_amount: 1598.0

CHECKING ROW CONTENT | row_2:
ContactName: Dick Terrcotta
cumulative_amount: 2022.0

CHECKING ROW CONTENT | row_3:
ContactName: Jack Diamond III
cumulative_amount: 2193.0

CHECKING ROW CONTENT | row_4:
ContactName: Barbara Berber
cumulative_amount: 5156.0

CHECKING ROW CONTENT | row_5:
ContactName: Toni Faucet
cumulative_amount: 8700.0


## Obtaining  the employee  who sells the most

In [8]:
def best_employee(db_cursor):
    """Implement the best_employee method to determine who’s the best employee! By “best employee”, we mean the one who
       sells the most.
       We expect the function to return a tuple like: ('FirstName', 'LastName', 6000 (the sum of all purchase)). The order
       of the information is irrelevant"""
    
    query = '''
        SELECT
            Employees.FirstName,
            Employees.LastName,
            SUM(details.UnitPrice * details.Quantity) AS cumulative_amount
        FROM OrderDetails AS details
        JOIN Orders ON details.OrderID = Orders.OrderID
        JOIN Employees ON Employees.EmployeeID = Orders.EmployeeID
        GROUP BY Employees.EmployeeID
        ORDER BY cumulative_amount DESC
        LIMIT 1
    '''
    
    rows = db_cursor.execute(query).fetchall()
    print(f"""
{type(rows) = } / {len(rows) = }
{type(rows) = }
{[row.keys()for row in rows][0] = }
""")

    orders_table_keys = [row.keys() for row in rows][0]

    for index, row in enumerate(rows[::]):
        print(f"""
CHECKING ROW CONTENT | row_{index + 1}:""")
        for key in orders_table_keys:
            print(f"""{key}: {row[key]}""")

    return rows

best_employee(db_cursor= cursor)


type(rows) = <class 'list'> / len(rows) = 1
type(rows) = <class 'list'>
[row.keys()for row in rows][0] = ['FirstName', 'LastName', 'cumulative_amount']


CHECKING ROW CONTENT | row_1:
FirstName: Patty
LastName: Lee
cumulative_amount: 7945.6


## Obtaining  a list with customers info depending on number of orders

In [9]:
def orders_per_customer(db_cursor):
    """Return a list of tuples where each tuple contains the contactName
    of the customer and the number of orders they made (contactName,
    number_of_orders). Order the list by ascending number of orders"""
    
    query = '''
            SELECT
            Customers.ContactName,
            COUNT(Orders.OrderID) AS order_amount
            FROM Customers
            LEFT JOIN Orders ON Customers.CustomerID = Orders.CustomerID
            GROUP BY Customers.CustomerID
            ORDER BY order_amount ASC
            '''
    
    rows = db_cursor.execute(query).fetchall()
    
    print(f"""
{type(rows) = } / {len(rows) = }
{type(rows) = }
{[row.keys()for row in rows][0] = }
""")

    orders_table_keys = [row.keys() for row in rows][0]

    for index, row in enumerate(rows[::]):
        print(f"""
CHECKING ROW CONTENT | row_{index + 1}:""")
        for key in orders_table_keys:
            print(f"""{key}: {row[key]}""")

    return rows

orders_per_customer(db_cursor= cursor)


type(rows) = <class 'list'> / len(rows) = 6
type(rows) = <class 'list'>
[row.keys()for row in rows][0] = ['ContactName', 'order_amount']


CHECKING ROW CONTENT | row_1:
ContactName: Sebastien Saunier
order_amount: 0

CHECKING ROW CONTENT | row_2:
ContactName: Jack Diamond III
order_amount: 2

CHECKING ROW CONTENT | row_3:
ContactName: Dick Terrcotta
order_amount: 3

CHECKING ROW CONTENT | row_4:
ContactName: Toni Faucet
order_amount: 4

CHECKING ROW CONTENT | row_5:
ContactName: Barbara Berber
order_amount: 5

CHECKING ROW CONTENT | row_6:
ContactName: Jim Wood
order_amount: 6


## Obtaining  a ranking of customers according to the order date

In [10]:
def order_rank_per_customer(db_cursor):
    """
    Implement order_rank_per_customer to rank the orders of each customer according to the order date.
    For each customer, the orders should be ranked in the chronological order.
    This function should return a list of tuples like (OrderID, CustomerID, OrderDate, OrderRank).
    """
    query = """
            SELECT
                OrderID,    
                CustomerID,
                OrderDate,
                RANK() OVER (
                            PARTITION BY CustomerID
                            ORDER BY OrderDate
                            ) AS OrderRank
            FROM Orders
            """
    rows = db_cursor.execute(query).fetchall()
    print(f"""
{type(rows) = } / {len(rows) = }
{type(rows) = }
{[row.keys()for row in rows][0] = }
""")

    orders_table_keys = [row.keys() for row in rows][0]

    for index, row in enumerate(rows[::]):
        print(f"""
CHECKING ROW CONTENT | row_{index + 1}:""")
        for key in orders_table_keys:
            print(f"""{key}: {row[key]}""")

    return rows

order_rank_per_customer(db_cursor=cursor)


type(rows) = <class 'list'> / len(rows) = 20
type(rows) = <class 'list'>
[row.keys()for row in rows][0] = ['OrderID', 'CustomerID', 'OrderDate', 'OrderRank']


CHECKING ROW CONTENT | row_1:
OrderID: 1
CustomerID: 1
OrderDate: 2012-01-04
OrderRank: 1

CHECKING ROW CONTENT | row_2:
OrderID: 8
CustomerID: 1
OrderDate: 2012-06-13
OrderRank: 2

CHECKING ROW CONTENT | row_3:
OrderID: 12
CustomerID: 1
OrderDate: 2012-09-13
OrderRank: 3

CHECKING ROW CONTENT | row_4:
OrderID: 2
CustomerID: 2
OrderDate: 2012-01-27
OrderRank: 1

CHECKING ROW CONTENT | row_5:
OrderID: 4
CustomerID: 2
OrderDate: 2012-03-13
OrderRank: 2

CHECKING ROW CONTENT | row_6:
OrderID: 9
CustomerID: 2
OrderDate: 2012-07-06
OrderRank: 3

CHECKING ROW CONTENT | row_7:
OrderID: 14
CustomerID: 2
OrderDate: 2012-10-29
OrderRank: 4

CHECKING ROW CONTENT | row_8:
OrderID: 19
CustomerID: 2
OrderDate: 2013-02-21
OrderRank: 5

CHECKING ROW CONTENT | row_9:
OrderID: 6
CustomerID: 3
OrderDate: 2012-04-28
OrderRank: 1

CHECKING ROW CONT

## Obtaining the cumulative amount of the orders of each customer according to order date

In [11]:
def order_cumulative_amount_per_customer(db_cursor):
    """
    Implement order_cumulative_amount_per_customer to compute the cumulative amount (in USD) of the orders of each 
    customer according to the order date.
    For each customer, the orders should be ranked in the chronological order.
    This function should return a list of tuples like (OrderID, CustomerID, OrderDate, OrderCumulativeAmount).
    """
    query = """
        SELECT
            Orders.OrderID,
            Orders.CustomerID,
            Orders.OrderDate,
            SUM(SUM(OrderDetails.UnitPrice * OrderDetails.Quantity)) OVER(PARTITION BY Orders.CustomerID ORDER BY Orders.OrderDate) OrderCumulativeAmount
        FROM Orders
        JOIN OrderDetails ON Orders.OrderID = OrderDetails.OrderID
        GROUP BY Orders.OrderID
        ORDER BY Orders.CustomerID
    """

    rows = db_cursor.execute(query).fetchall()
    print(f"""
{type(rows) = } / {len(rows) = }
{type(rows) = }
{[row.keys()for row in rows][0] = }
    """)

    orders_table_keys = [row.keys() for row in rows][0]

    for index, row in enumerate(rows[::]):
        print(f"""
CHECKING ROW CONTENT | row_{index + 1}:""")
        for key in orders_table_keys:
            print(f"""{key}: {row[key]}""")

    return rows

order_cumulative_amount_per_customer(db_cursor= cursor)


type(rows) = <class 'list'> / len(rows) = 20
type(rows) = <class 'list'>
[row.keys()for row in rows][0] = ['OrderID', 'CustomerID', 'OrderDate', 'OrderCumulativeAmount']
    

CHECKING ROW CONTENT | row_1:
OrderID: 1
CustomerID: 1
OrderDate: 2012-01-04
OrderCumulativeAmount: 48.0

CHECKING ROW CONTENT | row_2:
OrderID: 8
CustomerID: 1
OrderDate: 2012-06-13
OrderCumulativeAmount: 1989.7

CHECKING ROW CONTENT | row_3:
OrderID: 12
CustomerID: 1
OrderDate: 2012-09-13
OrderCumulativeAmount: 2021.7

CHECKING ROW CONTENT | row_4:
OrderID: 2
CustomerID: 2
OrderDate: 2012-01-27
OrderCumulativeAmount: 1948.7000000000003

CHECKING ROW CONTENT | row_5:
OrderID: 4
CustomerID: 2
OrderDate: 2012-03-13
OrderCumulativeAmount: 2348.7000000000003

CHECKING ROW CONTENT | row_6:
OrderID: 9
CustomerID: 2
OrderDate: 2012-07-06
OrderCumulativeAmount: 2648.7000000000003

CHECKING ROW CONTENT | row_7:
OrderID: 14
CustomerID: 2
OrderDate: 2012-10-29
OrderCumulativeAmount: 3529.7000000000003

CHECKING ROW CONTEN

## Obtaining average amount spent per order for each customer

In [12]:
def get_average_purchase(db_cursor):
    """return the average amount spent per order for each customer ordered by customer ID"""
    query = '''
        WITH OrderValues AS (
          SELECT
            SUM(od.UnitPrice * od.Quantity) AS value,
            od.OrderID
          FROM OrderDetails od
          GROUP BY od.OrderID
        )
        SELECT
            c.CustomerID,
            ROUND(AVG(ov.value), 2) AS average
        FROM Customers c
        JOIN Orders o ON c.CustomerID = o.CustomerID
        JOIN OrderValues ov ON ov.OrderID = o.OrderID
        GROUP BY c.CustomerID
        ORDER BY c.CustomerID
    '''
    rows = db_cursor.execute(query).fetchall()
    print(f"""
        {type(rows) = } / {len(rows) = }
        {type(rows) = }
        {[row.keys()for row in rows][0] = }
        """)

    orders_table_keys = [row.keys() for row in rows][0]

    for index, row in enumerate(rows[::]):
        print(f"""
CHECKING ROW CONTENT | row_{index + 1}:""")
        for key in orders_table_keys:
            print(f"""{key}: {row[key]}""")
            
    return db_cursor.execute(query).fetchall()

get_average_purchase(db_cursor=cursor)


        type(rows) = <class 'list'> / len(rows) = 5
        type(rows) = <class 'list'>
        [row.keys()for row in rows][0] = ['CustomerID', 'average']
        

CHECKING ROW CONTENT | row_1:
CustomerID: 1
average: 673.9

CHECKING ROW CONTENT | row_2:
CustomerID: 2
average: 1031.24

CHECKING ROW CONTENT | row_3:
CustomerID: 3
average: 266.32

CHECKING ROW CONTENT | row_4:
CustomerID: 4
average: 2175.03

CHECKING ROW CONTENT | row_5:
CustomerID: 5
average: 1096.3


## Obtaining the average amount spent per order

In [13]:
def get_general_avg_order(db_cursor):
    """return the average amount spent per order"""

    query = '''
        WITH OrderValues AS (
          SELECT SUM(od.Quantity * od.UnitPrice) AS value
          FROM OrderDetails od
          GROUP BY od.OrderID
        )
        SELECT ROUND(AVG(ov.value), 2) AS AverageAmountOrder
        FROM OrderValues ov
    '''
    rows = db_cursor.execute(query).fetchall()
    print(f"""
{type(rows) = } / {len(rows) = }
{type(rows) = }
{[row.keys()for row in rows][0] = }
""")

    orders_table_keys = [row.keys() for row in rows][0]

    for index, row in enumerate(rows[::]):
        print(f"""
CHECKING ROW CONTENT | row_{index + 1}:""")
        for key in orders_table_keys:
            print(f"""{key}: {row[key]}""")
    return db_cursor.execute(query).fetchall()

get_general_avg_order(db_cursor=cursor)


type(rows) = <class 'list'> / len(rows) = 1
type(rows) = <class 'list'>
[row.keys()for row in rows][0] = ['AverageAmountOrder']


CHECKING ROW CONTENT | row_1:
AverageAmountOrder: 983.43


## Obtaining the best customers


In [14]:
def best_customers(db_cursor):
    """return the customers who have an average purchase greater than the general average purchase"""

    query = '''
        WITH OrderValues AS (
          SELECT
            SUM(od.UnitPrice * od.Quantity) AS value,
            od.OrderID
          FROM OrderDetails od
          GROUP BY od.OrderID
        ),
        GeneralOrderValue AS (
          SELECT ROUND(AVG(ov.value), 2) AS average
          FROM OrderValues ov
        )
        SELECT
          c.CustomerID,
          ROUND(AVG(ov.value),2) AS avg_amount_per_customer
        FROM Customers c
        JOIN Orders o ON o.CustomerID = c.CustomerID
        JOIN OrderValues ov ON ov.OrderID = o.OrderID
        GROUP BY c.CustomerID
        HAVING AVG(ov.value) > (SELECT average FROM GeneralOrderValue)
        ORDER BY avg_amount_per_customer DESC
    '''
    rows = db_cursor.execute(query).fetchall()
    print(f"""
{type(rows) = } / {len(rows) = }
{type(rows) = }
{[row.keys()for row in rows][0] = }
""")

    orders_table_keys = [row.keys() for row in rows][0]

    for index, row in enumerate(rows[::]):
        print(f"""
CHECKING ROW CONTENT | row_{index + 1}:""")
        for key in orders_table_keys:
            print(f"""{key}: {row[key]}""")
            
    return db_cursor.execute(query).fetchall()

best_customers(db_cursor= cursor)


type(rows) = <class 'list'> / len(rows) = 3
type(rows) = <class 'list'>
[row.keys()for row in rows][0] = ['CustomerID', 'avg_amount_per_customer']


CHECKING ROW CONTENT | row_1:
CustomerID: 4
avg_amount_per_customer: 2175.03

CHECKING ROW CONTENT | row_2:
CustomerID: 5
avg_amount_per_customer: 1096.3

CHECKING ROW CONTENT | row_3:
CustomerID: 2
avg_amount_per_customer: 1031.24


## Obtaining the list of the top ordered product by each customer

In [15]:
def top_ordered_product_per_customer(db_cursor):
    """return the list of the top ordered product by each customer
    based on the total ordered amount in USD"""
    query = """
        WITH OrderedProducts AS (
            SELECT
                CustomerID,
                ProductID, SUM(OrderDetails.Quantity * OrderDetails.UnitPrice) AS ProductValue
            FROM OrderDetails
            JOIN Orders ON OrderDetails.OrderID = Orders.OrderID
            GROUP BY Orders.CustomerID, OrderDetails.ProductID
            ORDER BY ProductValue DESC
        ),
        ranks AS (
        SELECT
            OrderedProducts.CustomerID,
            OrderedProducts.ProductID,
            OrderedProducts.ProductValue,
            RANK() OVER(PARTITION BY OrderedProducts.CustomerID ORDER BY OrderedProducts.ProductValue DESC) as order_rank
            FROM OrderedProducts
            )
        SELECT ranks.CustomerID,ranks.ProductID, ranks.ProductValue
        from ranks
        WHERE order_rank = 1
        ORDER BY ranks.ProductValue DESC
    """
    rows = db_cursor.execute(query).fetchall()
    print(f"""
{type(rows) = } / {len(rows) = }
{type(rows) = }
{[row.keys()for row in rows][0] = }
""")

    orders_table_keys = [row.keys() for row in rows][0]

    for index, row in enumerate(rows[::]):
        print(f"""
CHECKING ROW CONTENT | row_{index + 1}:""")
        for key in orders_table_keys:
            print(f"""{key}: {row[key]}""")
            
    return db_cursor.execute(query).fetchall()

top_ordered_product_per_customer(db_cursor=cursor)


type(rows) = <class 'list'> / len(rows) = 5
type(rows) = <class 'list'>
[row.keys()for row in rows][0] = ['CustomerID', 'ProductID', 'ProductValue']


CHECKING ROW CONTENT | row_1:
CustomerID: 4
ProductID: 6
ProductValue: 5876.0

CHECKING ROW CONTENT | row_2:
CustomerID: 2
ProductID: 5
ProductValue: 2791.6000000000004

CHECKING ROW CONTENT | row_3:
CustomerID: 1
ProductID: 6
ProductValue: 1909.7

CHECKING ROW CONTENT | row_4:
CustomerID: 3
ProductID: 3
ProductValue: 1200.0

CHECKING ROW CONTENT | row_5:
CustomerID: 5
ProductID: 6
ProductValue: 1175.2


## Obtaining the average number of days between two consecutive orders of the same customer

In [16]:
def average_number_of_days_between_orders(db_cursor):
    """return the average number of days between two consecutive orders of the same customer"""
    query = """
        WITH DatedOrders AS (
            SELECT
                CustomerID,
                OrderID,
                OrderDate,
                LAG(OrderDate, 1, 0) OVER (
                    PARTITION BY CustomerID
                    ORDER By OrderDate
                ) PreviousOrderDate
            FROM Orders
        )
        SELECT ROUND(AVG(JULIANDAY(OrderDate) - JULIANDAY(PreviousOrderDate))) AS delta
        FROM DatedOrders
        WHERE PreviousOrderDate != 0
    """
    rows = db_cursor.execute(query).fetchall()
    print(f"""
{type(rows) = } / {len(rows) = }
{type(rows) = }
{[row.keys()for row in rows][0] = }
""")

    orders_table_keys = [row.keys() for row in rows][0]

    for index, row in enumerate(rows[::]):
        print(f"""
CHECKING ROW CONTENT | row_{index + 1}:""")
        for key in orders_table_keys:
            print(f"""{key}: {row[key]}""")
    return db_cursor.execute(query).fetchall()

average_number_of_days_between_orders(db_cursor=cursor)


type(rows) = <class 'list'> / len(rows) = 1
type(rows) = <class 'list'>
[row.keys()for row in rows][0] = ['delta']


CHECKING ROW CONTENT | row_1:
delta: 89.0


## Creating and exporting 2 csv to work in Power BI (All tables in the database)

In [17]:
def exporting_csv(db_cursor):
    query_exploring = """
            SELECT name FROM sqlite_master  
            WHERE type='table';
            """

    cursor.execute(query_exploring)
    rows = cursor.fetchall()
    print(rows)

    for index, row in enumerate(rows[1::]):
        print(f"""{index}_{row[0]}""")
        query_table = f"""
                SELECT * FROM {row[0]}  
                """
        print(query_table)
        movies = pd.read_sql(query_table, connection)
        movies.to_csv(f"data/csv_ecommerce_{row[0]}.csv", index=False)

exporting_csv(db_cursor=cursor)

[<sqlite3.Row object at 0x7f7b5264e650>, <sqlite3.Row object at 0x7f7b52699630>, <sqlite3.Row object at 0x7f7b52698790>, <sqlite3.Row object at 0x7f7b526984c0>, <sqlite3.Row object at 0x7f7b526996f0>]
0_Shippers

                SELECT * FROM Shippers  
                
1_Employees

                SELECT * FROM Employees  
                
2_Orders

                SELECT * FROM Orders  
                
3_OrderDetails

                SELECT * FROM OrderDetails  
                
